In [34]:
import math
import numpy as np
import pandas as pd
import random

def sigmoid(x):
    # 시그모이드 함수 정의
    y = 1 / (1 + np.exp(-x))
    return y

def sigmoid_list(x_list):
    # 다차원 배열에 대한 시그모이드 함수 적용
    y = np.zeros((len(x_list), 3))
    for i in range(len(x_list)):
        for j in range(3):
            y[i][j] = sigmoid(x_list[i][j])
    return y

def zeta(chromo, x1, x2, x3):
    # z 값을 계산하는 함수 정의
    z = []
    chromosome = pd.DataFrame(chromo)
    w = np.array(chromosome.iloc[:, :3])  # 앞의 3행이 가중치 행렬
    b = np.array(chromosome.iloc[:, 3:])  # 뒤의 1행이 편차
    for idx in range(len(x1)):
        # 각 유전자에 대해 가중치 배열 chromo[:3]*[x1,x2,x3]^-1+b(choromo[3])을 이용하여 z값을 받음
        z.append(np.dot(w, np.array([x1[idx], x2[idx], x3[idx]])) + b)
    return z

def top_found(fitness, chromosome):
    # 최대 적합도의 염색체를 반환하는 함수 정의
    print(f'Max fitness: {np.max(fitness)}')
    return chromosome[np.argmax(fitness)]

def selection(chromosome, fitness):
    # 전체 중 상위 10%의 결과만 선택
    num_top = int(len(chromosome) * 0.1)
    selected_list = []
    
    # 상위 10%의 인덱스를 찾아 선택
    top_indices = np.argsort(fitness)[-num_top:]
    selected_list = [chromosome[i] for i in top_indices]
    
    return selected_list

def crossover(chromosome):
    # 더 복잡한 교차 연산 적용 (한 지점 교차)
    crossover_list = []
    while len(crossover_list) < len(chromosome):
        idx1 = np.random.randint(len(chromosome))
        idx2 = np.random.randint(len(chromosome))
        rand_num = np.random.rand(1)
        crossover_point = np.random.randint(3)  # 3x3 행렬이므로 랜덤으로 하나의 행/열 선택
        
        # 교차 연산 적용
        crossover_list.append(np.concatenate((chromosome[idx1][:crossover_point],
                                              chromosome[idx2][crossover_point:])))
        crossover_list.append(np.concatenate((chromosome[idx2][:crossover_point],
                                              chromosome[idx1][crossover_point:])))
    return crossover_list

def mutation(chromosome):
    # 더 다양한 돌연변이 연산 적용
    for i in range(len(chromosome)):
        for j in range(3):
            # 돌연변이 적용 확률을 낮춤
            if np.random.rand() < 0.1:
                # 난수 범위 확대 및 변경
                chromosome[i][j] += random.uniform(-1, 1)
                chromosome[i][j][3] += random.uniform(-20, 20)
    return chromosome

def get_fitness(chromosome, x1, x2, x3):
    # 적합도를 계산하는 함수 정의
    fitness = []
    for i in range(len(chromosome)):
        fit = (1 - np.average(np.abs(np_label - sigmoid_list(zeta(chromosome[i], x1, x2, x3))))) * 100
        fitness.append(round(fit, 3))
    return fitness

def neural_network(w1, w2, b1, b2, data):
    # 신경망 출력 계산 함수 정의
    net_results = []
    for i in range(len(data)):
        net_results.append(np.dot(w2, np.dot(w1, data[i]) + b1.reshape(18,)) + b2.reshape(3,))
    return net_results
def test_results(max_chromosome, test_input):
    # 테스트 결과를 반환하는 함수 정의
    test_network = neural_network(init_W1, init_W2, init_b1, init_b2, test_input)
    tn = pd.DataFrame(test_network)
    t1 = np.array(tn.iloc[:, 0:1])
    t2 = np.array(tn.iloc[:, 1:2])
    t3 = np.array(tn.iloc[:, 2:3])
    prediction_list = []

    
    predictions = sigmoid_list(zeta(max_chromosome, t1, t2, t3))

    for prediction in predictions:
        prediction_list.append([round(val) for val in prediction])

    return prediction_list


# 상수
population = 50
df_label = pd.read_csv('p2_training_data_label.csv')
df_data = pd.read_csv('p2_training_data_mnist.csv')
df_test = pd.read_csv('p2_test_data_mnist.csv')
np_label = np.array(df_label)
np_data = np.array(df_data)
np_test = np.array(df_test)

# 초기 가중치 및 편향
init_W1 = np.zeros((18, 784))
init_W2 = np.zeros((3, 18))
init_b1 = np.zeros((18, 1))
init_b2 = np.zeros((3, 1))

# 초기 염색체 집단
initial_population = []
for j in range(population):
    ini = np.zeros((3, 4))
    for i in range(3):
        for k in range(3):
            ini[i][k] = random.uniform(-1, 1)
        ini[i][-1] = random.uniform(-50, 50)
    initial_population.append(ini)
initial_population = np.array(initial_population)

# 가중치 및 편향의 무작위 초기화
for i in range(len(init_W1)):
    for j in range(len(init_W1[0])):
        init_W1[i][j] = random.uniform(-1, 1)
    init_b1[i] = random.uniform(-100, 100)

for i in range(len(init_W2)):
    for j in range(len(init_W2[0])):
        init_W2[i][j] = random.uniform(-1, 1)
    init_b2[i] = random.uniform(-100, 100)

net_result = neural_network(init_W1, init_W2, init_b1, init_b2, np_data)
nr = pd.DataFrame(net_result)

x1 = np.array(nr.iloc[:, 0:1])
x2 = np.array(nr.iloc[:, 1:2])
x3 = np.array(nr.iloc[:, 2:3])

chromosomes = initial_population
max_chromosome = []

# 30세대 동안 진화 프로세스 실행
for i in range(30):
    chromosomes = selection(chromosomes, get_fitness(chromosomes, x1, x2, x3))
    chromosomes = crossover(chromosomes)
    chromosomes = mutation(chromosomes)
    max_chromosome = top_found(get_fitness(chromosomes, x1, x2, x3), chromosomes)

# 테스트 및 결과 출력
print_test_results(test_results(max_chromosome, np_test))


Max fitness: 66.667
Max fitness: 66.667
Max fitness: 66.667
Max fitness: 66.667
Max fitness: 66.667
Max fitness: 66.667
Max fitness: 66.667
Max fitness: 66.667
Max fitness: 66.667
Max fitness: 66.667
Max fitness: 66.667
Max fitness: 66.667
Max fitness: 66.667
Max fitness: 55.556
Max fitness: 55.556


C:\Users\kgm97\AppData\Local\Temp\ipykernel_100460\3913925453.py:8: RuntimeWarning: overflow encountered in exp
  y = 1 / (1 + np.exp(-x))


Max fitness: 66.667
Max fitness: 55.556
Max fitness: 44.444
Max fitness: 44.444
Max fitness: 44.444
Max fitness: 44.444
Max fitness: 44.444
Max fitness: 44.444
Max fitness: 44.444
Max fitness: 55.392
Max fitness: 55.392
Max fitness: 55.392
Max fitness: 44.444
Max fitness: 55.556
Max fitness: 55.556
Test Sample 1 - Predicted: 0, Actual: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.03921569 0.
 0.03137255 0.         0.04313725 0.         0.         0.01960784
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.03137255
 0.         0.         0.01568627 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         